# * Interim : 2025 Target Revenue TRUE
    96 Areas

## Parameter

In [2]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2025, 4, 25)

## ETL Process...

### Step 1 : Import Data Source

In [121]:
''' Rawdata '''

# Target_Revenue_TRUE_Y2025
src_file = '../../data/interim/Target_Revenue_TRUE_Y2025.xlsx'
src_df_cols = ['CUST_TYPE', 'TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'METRIC_VALUE', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_CD', 'AREA_DESC']

src_b2c_sheet = 'B2C Rawdata'
src_b2c_df = pd.read_excel(src_file, sheet_name=src_b2c_sheet, index_col=None) 
src_b2c_df = src_b2c_df.loc[src_b2c_df['AREA_TYPE']=="HH"]
src_b2c_df['CUST_TYPE'] = 'B2C'
src_b2c_df = src_b2c_df[src_df_cols]

src_b2b_sheet = 'B2B Rawdata'
src_b2b_df = pd.read_excel(src_file, sheet_name=src_b2b_sheet, index_col=None) 
src_b2b_df['CUST_TYPE'] = 'B2B'
src_b2b_df = src_b2b_df[src_df_cols]

src_df = pd.concat([src_b2c_df, src_b2b_df])
src_df.rename(columns={'AREA_CD': 'AREA_KEY', 'METRIC_VALUE': 'TARGET_MTH'}, inplace=True)
src_df['COMP_CD'] = 'TRUE' #src_df['COMP_CD'].astype(str)
# src_df = src_df.replace(np.nan, None)
src_df = src_df.reset_index(drop=True)

print(f'\nsrc_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')
src_df#.tail(3)


src_df : 1925 rows, 10 columns


,CUST_TYPE,TM_KEY_MTH,METRIC_CD,METRIC_NAME,TARGET_MTH,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC
0,B2C,202501,TB1R000100,Prepaid Revenue : TMH,5.710683e+06,TRUE,T,HH,902033,ANG THONG
1,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.081782e+07,TRUE,T,HH,902034,CHAI NAT
2,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.663773e+07,TRUE,T,HH,902035,KANCHANABURI
3,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.987740e+07,TRUE,T,HH,902036,LOP BURI
4,B2C,202501,TB1R000100,Prepaid Revenue : TMH,3.576186e+07,TRUE,T,HH,902037,NAKHON PATHOM
...,...,...,...,...,...,...,...,...,...,...
1920,B2B,202501,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide
1921,B2B,202502,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide
1922,B2B,202503,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide
1923,B2B,202504,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide


In [122]:
''' Add Columns '''

def product_group(v_cd):
    cd = v_cd
    result = ''
    if re.search('B1', cd): result = 'Prepaid'
    elif re.search('B2', cd): result = 'Postpaid'
    elif re.search('B3', cd): result = 'TOL'
    elif re.search('B4', cd): result = 'TVS'
    else: result = 'Unknown' 
    return result

src_df['PRODUCT_GRP'] = src_df.apply(lambda x: product_group(x['METRIC_CD']), axis=1)
# src_df['FREQUENCY'] = np.where(src_df['PRODUCT_GRP']=='Prepaid', 'DAY', 'BILL')
src_df['FREQUENCY'] = np.where(src_df['PRODUCT_GRP']=='Prepaid', 'Daily', 'Bill Cycle')
src_df['TM_KEY_YR'] = src_df['TM_KEY_MTH'].apply(str).str[:4].astype(int)

src_df#.tail(3)

,CUST_TYPE,TM_KEY_MTH,METRIC_CD,METRIC_NAME,TARGET_MTH,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,PRODUCT_GRP,FREQUENCY,TM_KEY_YR
0,B2C,202501,TB1R000100,Prepaid Revenue : TMH,5.710683e+06,TRUE,T,HH,902033,ANG THONG,Prepaid,Daily,2025
1,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.081782e+07,TRUE,T,HH,902034,CHAI NAT,Prepaid,Daily,2025
2,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.663773e+07,TRUE,T,HH,902035,KANCHANABURI,Prepaid,Daily,2025
3,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.987740e+07,TRUE,T,HH,902036,LOP BURI,Prepaid,Daily,2025
4,B2C,202501,TB1R000100,Prepaid Revenue : TMH,3.576186e+07,TRUE,T,HH,902037,NAKHON PATHOM,Prepaid,Daily,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,B2B,202501,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025
1921,B2B,202502,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025
1922,B2B,202503,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025
1923,B2B,202504,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025


In [123]:
''' Master Data '''

# DIM_TIME
dt_file = '../CFW/data/dim_time.csv'
# dt_cols = ['TM_KEY_YR', 'MONTH_SHORT', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH']
dt_cols = ['TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAY_NO', 'DAYS_IN_MONTH', 'PERIODFLAG']
dt_df = pd.read_csv(dt_file, usecols=dt_cols)

# DIM_MOOC_AREA
mooc_file = '../CFW/data/dim_mooc_area.csv'
mooc_cols = ['ZONE_TYPE', 'TEAM_CODE', 'ORGID_G', 'TDS_SGMD', 'ORGID_R', 'TDS_RGM_CODE', 'ORGID_H', 'HOP_HINT', 'TDS_PROVINCE', 'PROVINCE_ENG', 'PROVINCE_TH', 'ORGID_HH', 'D_CLUSTER', 'CCAATT', 'REMARK']
mooc_df = pd.read_csv(mooc_file, usecols=mooc_cols)
mooc_df = mooc_df.loc[(mooc_df['REMARK']!='Dummy') & (mooc_df['TEAM_CODE']!='ไม่ระบุ') & (mooc_df['HOP_HINT']!='True Corp')]

# Create HH level
mooc_hh_df = mooc_df[['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT', 'ORGID_HH', 'D_CLUSTER']].drop_duplicates()
mooc_hh_df.dropna(how='all', inplace=True)
mooc_hh_df['AREA_KEY'] = mooc_hh_df['ORGID_HH'].astype(int).astype(str)

In [124]:
''' Portion Data '''

portion_file = '../CFW/data/revenue_portion.xlsx'

new_existing_sheet = 'New & Existing'
new_existing_df = pd.read_excel(portion_file, sheet_name=new_existing_sheet, index_col=None) 
new_existing_df = new_existing_df.loc[new_existing_df['TM_KEY_YR']==2025]
new_existing_cols = ['PRODUCT_GRP', 'TM_KEY_MTH', 'NEW', 'EXIST']
new_existing_df = new_existing_df[new_existing_cols]
# new_existing_df

bill_cycle_sheet = 'Bill Cycle'
bill_cycle_df = pd.read_excel(portion_file, sheet_name=bill_cycle_sheet, index_col=None) 
bill_cycle_df = bill_cycle_df.loc[bill_cycle_df['TM_KEY_YR']==2025]
bill_cycle_cols = ['METRIC_CD', 'TM_KEY_YR', 'BILLING_DAY', 'BILL_PORTION']
bill_cycle_df = bill_cycle_df[bill_cycle_cols]
# bill_cycle_df

# src_df_cols = ['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'METRIC_VALUE', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_CD', 'AREA_DESC']
# src_df = src_df[src_df_cols]
# src_df.rename(columns={'AREA_CD': 'AREA_KEY', 'METRIC_VALUE': 'MTH_VALUE'}, inplace=True)

### Step 2 : Aggregate Data

In [7]:
# ''' Example DataFrame '''

# src_df.tail(3)
# dt_df.tail(3)
# mooc_df.tail(3)
# mooc_h_df.tail(3)
# mooc_h_df.loc[mooc_h_df['ORGID_H'].str.contains('^0')].tail(3)

In [130]:
''' Filter Rawdata

    TB1R000100	Prepaid Revenue : TMH
    TB2R010100	Postpaid Revenue B2C : TMH
    TB2R020100	Postpaid Revenue B2B : TMH
    TB3R000100	TOL Revenue
    TB4R000100	TVS Revenue
'''

''' Filter '''
raw_df = src_df.copy()
# raw_df = raw_df.loc[raw_df['TM_KEY_MTH']>=202504]
# raw_df = raw_df.loc[raw_df['TM_KEY_MTH'].isin([202501, 202502, 202503])]
# raw_df = raw_df.loc[raw_df['METRIC_CD'].isin(['TB1R000100', 'TB2R010100', 'TB3R000100', 'TB4R000100'])]

''' Data Test '''
# raw_df = raw_df.loc[raw_df['TM_KEY_MTH']==202504]
# raw_df = raw_df.loc[raw_df['METRIC_CD']=='TB1R000100']
# raw_df = raw_df.loc[raw_df['AREA_KEY']=='902033']
# raw_df = raw_df.loc[raw_df['AREA_KEY'].isna()]

raw_df = raw_df.reset_index(drop=True)
print(f'\nraw_df : {raw_df.shape[0]} rows, {raw_df.shape[1]} columns')
raw_df#.tail(3)


raw_df : 1925 rows, 13 columns


,CUST_TYPE,TM_KEY_MTH,METRIC_CD,METRIC_NAME,TARGET_MTH,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,PRODUCT_GRP,FREQUENCY,TM_KEY_YR
0,B2C,202501,TB1R000100,Prepaid Revenue : TMH,5.710683e+06,TRUE,T,HH,902033,ANG THONG,Prepaid,Daily,2025
1,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.081782e+07,TRUE,T,HH,902034,CHAI NAT,Prepaid,Daily,2025
2,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.663773e+07,TRUE,T,HH,902035,KANCHANABURI,Prepaid,Daily,2025
3,B2C,202501,TB1R000100,Prepaid Revenue : TMH,1.987740e+07,TRUE,T,HH,902036,LOP BURI,Prepaid,Daily,2025
4,B2C,202501,TB1R000100,Prepaid Revenue : TMH,3.576186e+07,TRUE,T,HH,902037,NAKHON PATHOM,Prepaid,Daily,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,B2B,202501,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025
1921,B2B,202502,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025
1922,B2B,202503,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025
1923,B2B,202504,TB2R020100,Postpaid Revenue B2B : TMH,3.182800e+08,TRUE,T,P,P,Nationwide,Postpaid,Bill Cycle,2025


In [131]:
''' Join New & Existing '''

merge_new_existing_df = pd.merge(raw_df, new_existing_df, how='left', on=['PRODUCT_GRP', 'TM_KEY_MTH'])
merge_new_existing_df['TARGET_MTH_NEW'] = merge_new_existing_df['TARGET_MTH'] * merge_new_existing_df['NEW']
merge_new_existing_df['TARGET_MTH_EXIST'] = merge_new_existing_df['TARGET_MTH'] * merge_new_existing_df['EXIST']
merge_new_existing_df = merge_new_existing_df[['CUST_TYPE', 'TM_KEY_YR', 'TM_KEY_MTH', 'PRODUCT_GRP', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST']]
merge_new_existing_df.tail(3)

,CUST_TYPE,TM_KEY_YR,TM_KEY_MTH,PRODUCT_GRP,FREQUENCY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST
1922,B2B,2025,202503,Postpaid,Bill Cycle,TB2R020100,Postpaid Revenue B2B : TMH,TRUE,T,P,P,Nationwide,318280000.0,1.355537e+07,3.045373e+08
1923,B2B,2025,202504,Postpaid,Bill Cycle,TB2R020100,Postpaid Revenue B2B : TMH,TRUE,T,P,P,Nationwide,318280000.0,1.824236e+07,2.997777e+08
1924,B2B,2025,202505,Postpaid,Bill Cycle,TB2R020100,Postpaid Revenue B2B : TMH,TRUE,T,P,P,Nationwide,318280000.0,2.379846e+07,2.941419e+08


In [133]:
''' Allocate Daily '''

day_df = merge_new_existing_df.loc[merge_new_existing_df['FREQUENCY']=='Daily']

day_df = pd.merge(day_df, dt_df, how='left', on='TM_KEY_MTH')

day_df['TARGET_DAY'] = day_df['TARGET_MTH'] / day_df['DAYS_IN_MONTH']
day_df['TARGET_DAY_NEW'] = day_df['TARGET_MTH_NEW'] / day_df['DAYS_IN_MONTH']
day_df['TARGET_DAY_EXIST'] = day_df['TARGET_MTH_EXIST'] / day_df['DAYS_IN_MONTH']

day_df = day_df[['CUST_TYPE', 'TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH', 'PERIODFLAG', 'PRODUCT_GRP', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST']]
day_df.tail(3)

,CUST_TYPE,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,DAYS_IN_MONTH,PERIODFLAG,PRODUCT_GRP,FREQUENCY,METRIC_CD,...,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
14493,B2C,2025,202505,2025022,20250529,31,N,Prepaid,Daily,TB1R000100,...,T,HH,906112,YALA,7.902092e+06,1.420572e+06,6.481520e+06,254906.193081,45824.91498,209081.278101
14494,B2C,2025,202505,2025022,20250530,31,N,Prepaid,Daily,TB1R000100,...,T,HH,906112,YALA,7.902092e+06,1.420572e+06,6.481520e+06,254906.193081,45824.91498,209081.278101
14495,B2C,2025,202505,2025022,20250531,31,EM,Prepaid,Daily,TB1R000100,...,T,HH,906112,YALA,7.902092e+06,1.420572e+06,6.481520e+06,254906.193081,45824.91498,209081.278101


In [134]:
''' Allocate Bill Cycle '''

bill_df = merge_new_existing_df.loc[merge_new_existing_df['FREQUENCY']=='Bill Cycle']
bill_df = pd.merge(bill_df, bill_cycle_df, how='left', on=['METRIC_CD', 'TM_KEY_YR'])

day_in_month_df = dt_df[['TM_KEY_MTH', 'DAYS_IN_MONTH']].drop_duplicates()
bill_df = pd.merge(bill_df, day_in_month_df, how='left', on='TM_KEY_MTH')
bill_df['BILLING_DAY'] = np.where(bill_df['BILLING_DAY']==1, bill_df['DAYS_IN_MONTH'], bill_df['BILLING_DAY'])

period_flag_df = dt_df[['TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAY_NO', 'PERIODFLAG']].drop_duplicates()
period_flag_df.rename(columns={'DAY_NO': 'BILLING_DAY'}, inplace=True)
bill_df = pd.merge(bill_df, period_flag_df, how='left', on=['TM_KEY_MTH', 'BILLING_DAY'])

bill_df['TARGET_DAY'] = bill_df['TARGET_MTH'] * bill_df['BILL_PORTION']
bill_df['TARGET_DAY_NEW'] = bill_df['TARGET_MTH_NEW'] * bill_df['BILL_PORTION']
bill_df['TARGET_DAY_EXIST'] = bill_df['TARGET_MTH_EXIST'] * bill_df['BILL_PORTION']

bill_df = bill_df[['CUST_TYPE', 'TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH', 'PERIODFLAG', 'PRODUCT_GRP', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST']]

''' Test '''
# bill_df = bill_df.loc[bill_df['PRODUCT_GRP']=='TOL']

bill_df.tail(3)

,CUST_TYPE,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,DAYS_IN_MONTH,PERIODFLAG,PRODUCT_GRP,FREQUENCY,METRIC_CD,...,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
14447,B2B,2025,202505,2025021,20250522,31,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.379846e+07,2.941419e+08,1.563996e+07,1.169433e+06,1.445384e+07
14448,B2B,2025,202505,2025021,20250525,31,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.379846e+07,2.941419e+08,3.578867e+07,2.675994e+06,3.307449e+07
14449,B2B,2025,202505,2025022,20250528,31,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.379846e+07,2.941419e+08,4.426796e+07,3.310007e+06,4.091071e+07


In [135]:
''' Concat Day & Bill '''

day_and_bill_df = pd.concat([day_df, bill_df])

''' Test '''
# day_and_bill_df = day_and_bill_df.loc[day_and_bill_df['PRODUCT_GRP']=='Postpaid']
# day_and_bill_df = day_and_bill_df.loc[day_and_bill_df['AREA_KEY'].isin(['P', '902033'])]

print(f'day_and_bill_df : {day_and_bill_df.shape[0]} rows, {day_and_bill_df.shape[1]} columns')
day_and_bill_df.tail(3)

day_and_bill_df : 28946 rows, 22 columns


,CUST_TYPE,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,DAYS_IN_MONTH,PERIODFLAG,PRODUCT_GRP,FREQUENCY,METRIC_CD,...,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
14447,B2B,2025,202505,2025021,20250522,31,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.379846e+07,2.941419e+08,1.563996e+07,1.169433e+06,1.445384e+07
14448,B2B,2025,202505,2025021,20250525,31,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.379846e+07,2.941419e+08,3.578867e+07,2.675994e+06,3.307449e+07
14449,B2B,2025,202505,2025022,20250528,31,N,Postpaid,Bill Cycle,TB2R020100,...,T,P,P,Nationwide,318280000.0,2.379846e+07,2.941419e+08,4.426796e+07,3.310007e+06,4.091071e+07


In [136]:
''' Join Area '''

merge_hh_df = pd.merge(day_and_bill_df, mooc_hh_df, how='left', on='AREA_KEY')
# merge_hh_df = pd.merge(day_and_bill_df.loc[day_and_bill_df['AREA_TYPE']=='HH'], mooc_hh_df, how='left', on='AREA_KEY')

''' Test '''
# merge_hh_df = merge_hh_df.loc[merge_hh_df['PRODUCT_GRP']=='TOL']
# merge_hh_df = merge_hh_df.loc[merge_hh_df['AREA_KEY'].isin(['P', '902033'])]
# merge_hh_df = merge_hh_df.loc[merge_hh_df['TM_KEY_DAY']==20250302]

merge_hh_df = merge_hh_df[['CUST_TYPE', 'TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH', 'PERIODFLAG', 'PRODUCT_GRP', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT', 'ORGID_HH', 'D_CLUSTER', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST']]
print(f'merge_hh_df : {merge_hh_df.shape[0]} rows, {merge_hh_df.shape[1]} columns')
merge_hh_df.tail(3)

merge_hh_df : 28946 rows, 29 columns


,CUST_TYPE,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,DAYS_IN_MONTH,PERIODFLAG,PRODUCT_GRP,FREQUENCY,METRIC_CD,...,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
28943,B2B,2025,202505,2025021,20250522,31,N,Postpaid,Bill Cycle,TB2R020100,...,NaN,NaN,NaN,NaN,318280000.0,2.379846e+07,2.941419e+08,1.563996e+07,1.169433e+06,1.445384e+07
28944,B2B,2025,202505,2025021,20250525,31,N,Postpaid,Bill Cycle,TB2R020100,...,NaN,NaN,NaN,NaN,318280000.0,2.379846e+07,2.941419e+08,3.578867e+07,2.675994e+06,3.307449e+07
28945,B2B,2025,202505,2025022,20250528,31,N,Postpaid,Bill Cycle,TB2R020100,...,NaN,NaN,NaN,NaN,318280000.0,2.379846e+07,2.941419e+08,4.426796e+07,3.310007e+06,4.091071e+07


In [137]:
''' Aggregate P, G, H, HH '''

# agg_cols = ['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'DAY_VALUE', 'MTH_VALUE'] # , 'FREQUENCY', 'REMARK'
agg_cols = ['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'FREQUENCY', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST']

# P : Nationwide
agg_p_df = merge_hh_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'FREQUENCY']).agg({'TARGET_MTH': 'sum', 'TARGET_MTH_NEW': 'sum', 'TARGET_MTH_EXIST': 'sum', 'TARGET_DAY': 'sum', 'TARGET_DAY_NEW': 'sum', 'TARGET_DAY_EXIST': 'sum'}).reset_index()
agg_p_df['AREA_NO'] = 1
agg_p_df['AREA_TYPE'] = 'P'
agg_p_df['AREA_CD'] = 'P'
agg_p_df['AREA_NAME'] = 'Nationwide'
agg_p_df = agg_p_df.loc[:, agg_cols]
# agg_p_df[agg_p_df['TM_KEY_DAY']==20240501]

# G : Region
agg_g_df = merge_hh_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'FREQUENCY', 'ORGID_G', 'TDS_SGMD']).agg({'TARGET_MTH': 'sum', 'TARGET_MTH_NEW': 'sum', 'TARGET_MTH_EXIST': 'sum', 'TARGET_DAY': 'sum', 'TARGET_DAY_NEW': 'sum', 'TARGET_DAY_EXIST': 'sum'}).reset_index()
agg_g_df['AREA_NO'] = 2
agg_g_df['AREA_TYPE'] = 'G'
agg_g_df.rename(columns={'ORGID_G': 'AREA_CD'}, inplace=True)
agg_g_df.rename(columns={'TDS_SGMD': 'AREA_NAME'}, inplace=True)
agg_g_df = agg_g_df.loc[:, agg_cols]
# agg_g_df[agg_g_df['TM_KEY_DAY']==20240501]

# H : HOP_HINT
agg_h_df = merge_hh_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'FREQUENCY', 'ORGID_H', 'HOP_HINT']).agg({'TARGET_MTH': 'sum', 'TARGET_MTH_NEW': 'sum', 'TARGET_MTH_EXIST': 'sum', 'TARGET_DAY': 'sum', 'TARGET_DAY_NEW': 'sum', 'TARGET_DAY_EXIST': 'sum'}).reset_index()
agg_h_df['AREA_NO'] = 3
agg_h_df['AREA_TYPE'] = 'H'
agg_h_df.rename(columns={'ORGID_H': 'AREA_CD'}, inplace=True)
agg_h_df.rename(columns={'HOP_HINT': 'AREA_NAME'}, inplace=True)
agg_h_df = agg_h_df.loc[:, agg_cols]
# agg_h_df[agg_h_df['TM_KEY_DAY']==20240501]

# HH : D_CLUSTER
agg_hh_df = merge_hh_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'FREQUENCY', 'ORGID_HH', 'D_CLUSTER']).agg({'TARGET_MTH': 'sum', 'TARGET_MTH_NEW': 'sum', 'TARGET_MTH_EXIST': 'sum', 'TARGET_DAY': 'sum', 'TARGET_DAY_NEW': 'sum', 'TARGET_DAY_EXIST': 'sum'}).reset_index()
agg_hh_df['AREA_NO'] = 4
agg_hh_df['AREA_TYPE'] = 'HH'
agg_hh_df['ORGID_HH'] = agg_hh_df['ORGID_HH'].astype(int).astype(str)
agg_hh_df.rename(columns={'ORGID_HH': 'AREA_CD'}, inplace=True)
agg_hh_df.rename(columns={'D_CLUSTER': 'AREA_NAME'}, inplace=True)
agg_hh_df = agg_hh_df.loc[:, agg_cols]
# agg_hh_df[agg_hh_df['TM_KEY_DAY']==20240601]

# Concat DataFrame
agg_all_area_df = pd.concat([agg_p_df, agg_g_df, agg_h_df, agg_hh_df], ignore_index=True)
# agg_all_area_df['AGG_TYPE'] = 'S'
# agg_all_area_df['FREQUENCY'] = np.where(agg_all_area_df['FREQUENCY']=='DAY', 'Daily', 'Bill Cycle')
# agg_all_area_df['REMARK'] = 'Allocate from 96 Cluster (HH level)'
# # agg_all_area_df['REMARK'] = agg_all_area_df['TM_KEY_MTH'].apply(lambda x: 'H Level 64 Province' if x>=202401 and x<=202403 else 'HH Level 96 Cluster')
# agg_all_area_df = agg_all_area_df[['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'AGG_TYPE', 'FREQUENCY', 'REMARK']]

print(f'agg_all_area_df : {agg_all_area_df.shape[0]} rows, {agg_all_area_df.shape[1]} columns')
# agg_all_area_df.loc[agg_all_area_df['TM_KEY_DAY']==20240601]
agg_all_area_df.tail(3)

agg_all_area_df : 50411 rows, 19 columns


,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,FREQUENCY,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
50408,2025,202505,2025022,20250531,TB4R000100,TVS Revenue,TRUE,T,4,HH,910096,SURIN,Bill Cycle,1.549526e+06,5998.439643,1.543528e+06,509281.053895,1971.500602,507309.553294
50409,2025,202505,2025022,20250531,TB4R000100,TVS Revenue,TRUE,T,4,HH,910097,UBON RATCHATHANI,Bill Cycle,2.453917e+06,9499.467535,2.444418e+06,806526.217801,3122.179613,803404.038188
50410,2025,202505,2025022,20250531,TB4R000100,TVS Revenue,TRUE,T,4,HH,910098,YASOTHON,Bill Cycle,5.514499e+05,2134.742359,5.493152e+05,181244.440714,701.623438,180542.817276


In [138]:
''' Aggregate TB2R000100 : Postpaid Revenue : TMH (Nationwide Only)'''

post_revenue_tmh_df = agg_all_area_df.loc[(agg_all_area_df['METRIC_CD'].isin(['TB2R010100', 'TB2R020100'])) & (agg_all_area_df['AREA_CD']=='P')]
post_revenue_tmh_df = post_revenue_tmh_df\
    .groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'FREQUENCY'])\
        .agg({'TARGET_MTH': 'sum', 'TARGET_MTH_NEW': 'sum', 'TARGET_MTH_EXIST': 'sum', 'TARGET_DAY': 'sum', 'TARGET_DAY_NEW': 'sum', 'TARGET_DAY_EXIST': 'sum'}).reset_index()
post_revenue_tmh_df['METRIC_CD'] = 'TB2R000100'
post_revenue_tmh_df['METRIC_NAME'] = 'Postpaid Revenue : TMH'
post_revenue_tmh_df = post_revenue_tmh_df[['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'FREQUENCY', 'TARGET_MTH', 'TARGET_MTH_NEW', 'TARGET_MTH_EXIST', 'TARGET_DAY', 'TARGET_DAY_NEW', 'TARGET_DAY_EXIST']]

''' Test '''
# post_revenue_tmh_df = post_revenue_tmh_df.loc[post_revenue_tmh_df['PRODUCT_GRP']=='TOL']
# post_revenue_tmh_df = post_revenue_tmh_df.loc[post_revenue_tmh_df['AREA_KEY'].isin(['P', '902033'])]
# post_revenue_tmh_df = post_revenue_tmh_df.loc[post_revenue_tmh_df['TM_KEY_MTH']==202503]
# post_revenue_tmh_df = post_revenue_tmh_df.loc[post_revenue_tmh_df['TM_KEY_DAY']==20250302]
# post_revenue_tmh_df = post_revenue_tmh_df.loc[post_revenue_tmh_df['AREA_TYPE']=='P']

print(f'post_revenue_tmh_df : {post_revenue_tmh_df.shape[0]} rows, {post_revenue_tmh_df.shape[1]} columns')
post_revenue_tmh_df.tail(3)

post_revenue_tmh_df : 49 rows, 19 columns


,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,FREQUENCY,TARGET_MTH,TARGET_MTH_NEW,TARGET_MTH_EXIST,TARGET_DAY,TARGET_DAY_NEW,TARGET_DAY_EXIST
46,2025,202505,2025021,20250525,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,Bill Cycle,3.676556e+09,2.749038e+08,3.397729e+09,3.677501e+08,2.749744e+07,3.398603e+08
47,2025,202505,2025022,20250528,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,Bill Cycle,3.676556e+09,2.749038e+08,3.397729e+09,3.957438e+08,2.959059e+07,3.657309e+08
48,2025,202505,2025022,20250531,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,Bill Cycle,3.676556e+09,2.749038e+08,3.397729e+09,2.585829e+07,1.933478e+06,2.389722e+07


In [139]:
''' Prepairing Latest Results '''

latest_concat_df = pd.concat([agg_all_area_df, post_revenue_tmh_df], ignore_index=True)
latest_concat_df.rename(columns={'TARGET_DAY': 'DAY_VALUE', 'TARGET_MTH': 'MTH_VALUE'}, inplace=True)
latest_concat_df['AGG_TYPE'] = 'S'
latest_concat_df['REMARK'] = 'Allocate from 96 Cluster (HH level)'

latest_results_df = latest_concat_df[['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'DAY_VALUE', 'MTH_VALUE', 'AGG_TYPE', 'FREQUENCY', 'REMARK']]
latest_results_df.tail(3)

,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,DAY_VALUE,MTH_VALUE,AGG_TYPE,FREQUENCY,REMARK
50457,2025,202505,2025021,20250525,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,3.677501e+08,3.676556e+09,S,Bill Cycle,Allocate from 96 Cluster (HH level)
50458,2025,202505,2025022,20250528,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,3.957438e+08,3.676556e+09,S,Bill Cycle,Allocate from 96 Cluster (HH level)
50459,2025,202505,2025022,20250531,TB2R000100,Postpaid Revenue : TMH,TRUE,T,1,P,P,Nationwide,2.585829e+07,3.676556e+09,S,Bill Cycle,Allocate from 96 Cluster (HH level)


In [140]:
''' Sample Data '''

chk_df = latest_results_df.loc[latest_results_df['TM_KEY_DAY']==20250402]

chk_df = chk_df\
    .groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE'])\
        .agg({'MTH_VALUE': 'sum'}).reset_index()

mod_col_list = chk_df.iloc[:, 8:].columns.tolist()
for col in mod_col_list:
    chk_df[col] = chk_df[col].apply(lambda x: format(x, ',.0f'))

chk_df

,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,FREQUENCY,METRIC_CD,METRIC_NAME,AREA_TYPE,MTH_VALUE
0,2025,202504,2025014,20250402,Bill Cycle,TB2R000100,Postpaid Revenue : TMH,P,"3,676,555,973"
1,2025,202504,2025014,20250402,Bill Cycle,TB2R010100,Postpaid Revenue B2C : TMH,G,"3,358,275,973"
2,2025,202504,2025014,20250402,Bill Cycle,TB2R010100,Postpaid Revenue B2C : TMH,H,"3,358,275,973"
3,2025,202504,2025014,20250402,Bill Cycle,TB2R010100,Postpaid Revenue B2C : TMH,HH,"3,358,275,973"
4,2025,202504,2025014,20250402,Bill Cycle,TB2R010100,Postpaid Revenue B2C : TMH,P,"3,358,275,973"
5,2025,202504,2025014,20250402,Bill Cycle,TB2R020100,Postpaid Revenue B2B : TMH,P,"318,280,000"
6,2025,202504,2025014,20250402,Bill Cycle,TB3R000100,TOL Revenue,G,"1,672,918,006"
7,2025,202504,2025014,20250402,Bill Cycle,TB3R000100,TOL Revenue,H,"1,672,918,006"
8,2025,202504,2025014,20250402,Bill Cycle,TB3R000100,TOL Revenue,HH,"1,672,918,006"
9,2025,202504,2025014,20250402,Bill Cycle,TB3R000100,TOL Revenue,P,"1,672,918,006"


In [ ]:
# main_kpi_df = merge_hh_df
# main_kpi_df = main_kpi_df[['CUST_TYPE', 'TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH', 'PERIODFLAG', 'PRODUCT_GRP', 'FREQUENCY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC', 'TARGET_MTH', 'TARGET_DAY']]

# main_kpi_df.tail(3)

### Step 3 : Insert to "INTERIM_VINSIGHT_DATA"
    Delete -> Insert

In [142]:
''' Input Parameter '''

# Create list
month_list = latest_results_df['TM_KEY_MTH'].drop_duplicates().tolist()
mt_cd_list = latest_results_df['METRIC_CD'].drop_duplicates().tolist()

if len(mt_cd_list) == 1:
    mt_cd_list = str(mt_cd_list).replace(r'[', '(').replace(r']', ')')
else:
    mt_cd_list = tuple(mt_cd_list)

# Create Param
# v_param = dict(mth_start=202406, mth_end=202408, metric_cd=mt_cd_list)
v_param = dict(mth_start=min(month_list), mth_end=max(month_list), metric_cd=mt_cd_list)
v_target_schema = 'AUTOKPI'
v_target_table = 'INTERIM_VINSIGHT_DATA'

# query_delete = f"DELETE {v_target_schema}.{v_target_table} WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} AND METRIC_CD IN {v_param['metric_cd']}"
query_delete = f"""
    DELETE {v_target_schema}.{v_target_table} 
    WHERE VERSION = 'T'
    AND METRIC_CD IN {v_param['metric_cd']}
    AND TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} 
"""

print(f"\nParameter...\n\n   -> TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']}\n   -> METRIC_CD IN {v_param['metric_cd']}")
print(f'\nDataFrame...\n\n   -> latest_results_df : {latest_results_df.shape[0]} rows, {latest_results_df.shape[1]} columns') 
print(f'\nquery_delete...\n{query_delete}')


Parameter...

   -> TM_KEY_MTH BETWEEN 202501 AND 202505
   -> METRIC_CD IN ('TB1R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100', 'TB2R000100')

DataFrame...

   -> latest_results_df : 50460 rows, 17 columns

query_delete...

    DELETE AUTOKPI.INTERIM_VINSIGHT_DATA 
    WHERE VERSION = 'T'
    AND METRIC_CD IN ('TB1R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100', 'TB2R000100')
    AND TM_KEY_MTH BETWEEN 202501 AND 202505 



In [143]:
# ''' DELEATE -> INSERT '''

# job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# print(f'\nJob Start... {job_start_datetime}')


# # Create rows from DataFrame
# rows = [tuple(x) for x in latest_results_df.values]


# # Connect : AKPIPRD
# dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
# conn = oracledb.connect(dsn)
# print(f'\n{AKPIPRD_db} : Connected')
# cur = conn.cursor()
# print(f'\nProcessing...')


# try:
#     # # Truncate
#     # cur.execute(f"TRUNCATE TABLE {v_target_schema}.{v_target_table}")
#     # print(f'\n   -> TRUNCATE : "{v_target_table}" : Done !')

#     # Delete
#     cur.execute(query_delete)
#     print(f'\n   -> DELETE : "{v_target_table}" : Done !')
    
#     # Insert
#     cur.executemany(f"""
#         INSERT INTO {v_target_table} 
#         (TM_KEY_YR, TM_KEY_MTH, TRUE_TM_KEY_WK, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, VERSION, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, DAY_VALUE, MTH_VALUE, AGG_TYPE, FREQUENCY, REMARK) 
#         VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17)
#         """, rows)
#     print(f'\n   -> INSERT : "{v_target_table}" : Done !')

#     cur.close()
#     conn.commit()


# except oracledb.DatabaseError as e:
#     print(f'\nError with Oracle : {e}')


# finally:
#     conn.close()
#     print(f'\n{AKPIPRD_db} : Disconnected')
#     print(f'\nJob Done !!!')



Job Start... 2025-04-25, 16:09:36

AKPIPRD : Connected

Processing...

   -> DELETE : "INTERIM_VINSIGHT_DATA" : Done !

   -> INSERT : "INTERIM_VINSIGHT_DATA" : Done !

AKPIPRD : Disconnected

Job Done !!!
